# agora_api

> SDK to interact with Agora private api

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.utils import *
import os
import httpx
from typing import Dict, Any, List
import json
from fewsats.core import Fewsats

In [ ]:
#| export

base_url = 'https://zues.searchagora.com/api/v1'

In [ ]:
#| export

#| export
class Agora:
    "Client for interacting with the Agora API"
    def __init__(self,
                 api_key: str = None, # The API key for the Agora account
                 base_url: str = "https://zues.searchagora.com/api/v1"): # The Agora API base URL
        self.api_key = api_key or os.environ.get("AGORA_API_KEY")
        # TODO: maybe change this for a decorator so search_trial can be used without api_key
        # if not self.api_key:
        #     raise ValueError("The api_key client option must be set either by passing api_key to the client or by setting the AGORA_API_KEY environment variable")
        self.base_url = base_url
        self._httpx_client = httpx.Client()
        self._httpx_client.headers.update({"Authorization": f"Bearer {self.api_key}"})



In [ ]:
a = Agora()

In [ ]:
#| export

@patch
def search_trial(self: Agora,
                 query: str, # Search query text
                price_min: int = 0, # Minimum price for filtering products
                price_max: int = None, # Maximum price for filtering products
                sort: str = None, # Sorting field: price:relevance
                order: str = None): # Sorting order: asc or desc
    """
    Search for products using the trial endpoint.
    
    Args:
        query (str): Search query text
        price_min (int, optional): Minimum price for filtering products (default: 0)
        price_max (int, optional): Maximum price for filtering products
        sort (str, optional): Sorting field: price:relevance
        order (str, optional): Sorting order: asc or desc
        
    Returns:
        dict: Search results with products matching the query
    
    Example:
        search_trial("shoes", [100, 1000], "price:relevance", "desc")
    """
    endpoint = f'{self.base_url}/search/trial'
    
    # Build query parameters
    params = {'q': query}

    # Handle price range parameters
    if price_min != 0 and price_max is not None:
        params['priceRange'] = [price_min, price_max]
        
    if sort: params['sort'] = sort
    if order: params['order'] = order
    
    # Make the request
    return httpx.get(endpoint, params=params)

In [ ]:

r = a.search_trial(sort= 'price:relevance', order='asc',query='glasses', price_max=1000)
p = r.json()['Products'][0]
p

{'name': 'Kaleidoscope Glasses',
 'storeName': 'Costumes, Etc...',
 'brand': 'Western Fashion',
 '_id': '677df599770698bbe867b39f',
 'slug': 'copy-of-kaleidscope-goggles-6b527f77-bd25-4846-ad2b-344fd2cc95ef-1736308121269',
 'price': 15,
 'isVerified': False,
 'isBoosted': False,
 'source': 'shopify',
 'images': ['https://cdn.shopify.com/s/files/1/0292/5599/1401/products/69132-4-1_1.jpg?v=1625761508'],
 'url': 'https://costumesetc.com/products/copy-of-kaleidscope-goggles',
 'agoraScore': 81,
 'priceHistory': [{'price': 15,
   'date': '2025-01-08T03:48:41.347Z',
   '_id': '677df5a3770698bbe867bcdc'}],
 'averageRating': 4.5,
 'discountVal': 0}

In [ ]:
json.dumps(p)

'{"name": "Headphones", "storeName": "The Baby Cubby", "brand": "Tonies", "_id": "6674b4aab49534201189688d", "slug": "tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056", "price": 29.99, "isVerified": false, "isBoosted": false, "source": "shopify", "images": ["https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-2.jpg?v=1637054132", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-3.jpg?v=1637054134", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/blue_8b41cdb7-1f12-4b3d-b43c-8aac607377f3.jpg?v=1637336366", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Pink-1.jpg?v=1637336366", "https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Reddd.jpg?v=1637336366"], "url": "https://babycubby.com/products/tonies-headphones", "agoraScore": 92, "priceHistory": 

## Methods

In [ ]:
#| export

@patch
def _request(self: Agora, 
             method: str, # The HTTP method to use
             path: str, # The path to request
             timeout: int = 10, # Timeout for the request in s
             **kwargs) -> Dict[str, Any]:
    "Makes an authenticated request to Agora API"
    url = f"{self.base_url}/{path}"
    return  self._httpx_client.request(method, url, timeout=timeout, **kwargs)


@patch
def text_search(self: Agora, 
                query: str, # Search query text
                count: int = 20, # Number of products per page (default: 20, max: 250)
                page: int = 1, # Page number for pagination (default: 1)
                price_min: int = 0, # Minimum price for filtering products
                price_max: int = None, # Maximum price for filtering products
                sort: str = None, # Sorting field: price:relevance
                order: str = None, # Sorting order: asc or desc
                image_id: str = None): # Image search identifier
    """
    Search for products with full functionality.
    
    Args:
        query (str): Search query text
        count (int, optional): Number of products per page (default: 20, max: 250)
        page (int, optional): Page number for pagination (default: 1)
        price_min (int, optional): Minimum price for filtering products (default: 0)
        price_max (int, optional): Maximum price for filtering products
        sort (str, optional): Sorting field: price:relevance
        order (str, optional): Sorting order: asc or desc
        image_id (str, optional): Image search identifier
        
    Returns:
        dict: Search results with products matching the query
    
    Example:
        text_search("red shoes", count=10, page=1, price_min=50, price_max=200, sort="price", order="asc")
    """
    
    # Build query parameters
    params = {'q': query, 'count': count, 'page': page}
    
    # Handle price range parameters
    if price_min != 0 and price_max is not None:
        params['priceRange'] = [price_min, price_max]
        
    if sort: params['sort'] = sort
    if order: params['order'] = order
    if image_id: params['imageId'] = image_id
    
    return self._request('GET', path='search', params=params)
    

In [ ]:
#| export

@patch
def get_product_detail(self: Agora, 
                      slug: str): # The unique identifier of the product to retrieve
    """
    Retrieve detailed information about a specific product.
    
    Args:
        slug (str): The unique identifier of the product to retrieve
        
    Returns:
        dict: Detailed information about the requested product
    
    Example:
        agora.get_product_detail("calzuro-without-pistachio-eb12f468-48a2-48af-9f5e-3fda5f6c135c-1708446961787")
    """
    # Build query parameters
    params = {'slug': slug}
    
    # Make the request
    return self._request('GET', path='product-detail', params=params)


In [ ]:

# get_product_detail(token, p.slug)

In [ ]:
#| export

@patch
def create_cart(self: Agora, 
                custom_user_id: str = None, # Unique identifier for the user
                items: List[Dict] = None): # List of items to add to the cart
    """
    Create a new cart for a user.
    
    Args:
        custom_user_id (str, optional): Unique identifier for the user
        items (list, optional): List of items to add to the cart
            Each item should be a dict with:
            - variantId (int): Variant ID of the product
            - product (int/str): Product ID
            - quantity (int): Quantity of the product
            
    Returns:
        dict: Response with cart creation status
    
    Example:
        agora.create_cart("user123", [
            {"variantId": 123, "product": "678f71a9356a36f784ee2e88", "quantity": 1}
        ])
    """
    # Add custom user ID if provided
    headers = {}
    if custom_user_id:
        headers['customuserid'] = custom_user_id
    
    # Prepare request body
    data = {}
    if items:
        data['items'] = items
    
    # Make the request
    return self._request('POST', path='cart', headers=headers, json=data)

In [ ]:
#| export

@patch
def add_to_cart(self: Agora, 
                product_id: str, # ID of the product to add
                variant_id: str, # Variant ID of the product
                quantity: int = 1, # Quantity of the product (default: 1)
                custom_user_id: str = None): # Unique identifier for the user
    """
    Add a product to an existing cart.
    
    Args:
        product_id (str/int): ID of the product to add
        variant_id (str/int): Variant ID of the product
        quantity (int, optional): Quantity of the product (default: 1)
        custom_user_id (str, optional): Unique identifier for the user
        
    Returns:
        dict: Response with cart update status
    
    Example:
        agora.add_to_cart("678f71a9356a36f784ee2e88", "2061038485517", 2, "user123")
    """
    # Add custom user ID if provided
    headers = {}
    if custom_user_id:
        headers['customuserid'] = custom_user_id
    
    # Prepare request body
    data = {
        "product": {
            "product": product_id,
            "variantId": variant_id,
            "quantity": quantity
        }
    }
    
    # Make the PUT request
    return self._request('PUT', path='cart', headers=headers, json=data)

In [ ]:
#| export

@patch
def create_order(self: Agora, 
                encrypted_payment_info: str, # Encrypted payment information
                shipping_address: Dict[str, str], # Dictionary containing shipping address details
                current_user: Dict[str, str]): # Dictionary containing user information
    """
    Create a new order from cart items.
    
    Args:
        encrypted_payment_info (str): Encrypted payment information
        shipping_address (dict): Dictionary containing shipping address details:
            - addressFirst (str): Street address
            - city (str): City name
            - state (str): State/province
            - country (str): Country name
            - addressName (str): Name associated with address
            - zipCode (str): Postal/ZIP code
        current_user (dict): Dictionary containing user information:
            - firstname (str): User's first name
            - lastname (str): User's last name
            - email (str): User's email address
            - _id (str): User's ID
            
    Returns:
        dict: Order creation response with success status and order ID
    
    Example:
        agora.create_order(
            "encrypted_data",
            {
                "addressFirst": "123 Main St",
                "city": "New York",
                "state": "NY",
                "country": "US",
                "addressName": "Home",
                "zipCode": "10001"
            },
            {
                "firstname": "John",
                "lastname": "Doe",
                "email": "john@example.com",
                "_id": "user123"
            }
        )
    """
    # Prepare request body
    data = {
        "encryptedPaymentInfo": encrypted_payment_info,
        "shippingAddress": shipping_address,
        "currentUser": current_user
    }
    
    # Make the POST request
    return self._request('POST', path='order', json=data)

In [ ]:
#| export

@patch
def track_order(self: Agora, 
               order_id: str): # Unique identifier of the order to track
    """
    Track an existing order by its ID.
    
    Args:
        order_id (str): Unique identifier of the order to track
        
    Returns:
        dict: Detailed order information including items, status, and shipping details
    
    Example:
        agora.track_order("67c8577b3e370f07d12c7722")
    """
    # Make the GET request
    return self._request('GET', path=f'order-tracking/{order_id}')

In [ ]:
#| export 

@patch
def refresh_token(self: Agora, 
                 refresh_token_str: str): # The refresh token to validate
    """
    Refresh API key and token by providing a valid refresh token.
    
    Args:
        refresh_token_str (str): The refresh token to validate and retrieve
                               a new API key and refresh token
        
    Returns:
        dict: Object containing new API key, refresh token, and expiration time
              with the following structure:
              {
                "status": "success",
                "data": {
                  "apiKey": "...",
                  "refreshToken": "...",
                  "expiresAt": "..."
                }
              }
    
    Raises:
        ValueError: If the refresh token is invalid or missing
    """
    # Prepare request body
    data = {"refreshToken": refresh_token_str}
    
    # Make the POST request (using direct httpx call since we don't need auth here)
    r = httpx.post(f'{self.base_url}/refresh-token', json=data)
    response = dict2obj(r.json())
    
    # Check for errors
    if r.status_code != 200 or response.status == "error":
        raise ValueError(response.message)
        
    return response

In [ ]:
#| export

@patch
def create_payment_intent(self: Agora,
                         offer_id: str, # Unique identifier for this offer (can be variant_id for items or user_id for carts)
                         amount: int, # Payment amount in cents
                         title: str, # Offer title
                         description: str, # Offer description
                         currency: str = "USD"):  # Payment currency
    """
    Create a payment intent for a product or cart.
    
    Args:
        offer_id (str): Unique identifier for this offer (variant_id for items, user_id for carts)
        amount (int): Payment amount in cents
        title (str): Offer title
        description (str): Offer description
        currency (str, optional): Payment currency (default: USD)
        
    Returns:
        dict: Created payment options for product / cart.
    
    Example:
        agora.create_payment_intent(
            "12345",
            1999, # 19.99 USD in cents
            title="Altra Shoes",
            description="Altra Escalanta v4 running shoes"
        )
    """
    
    # Create offer data
    offers_data = [{
        "offer_id": offer_id,
        "amount": amount,
        "currency": currency,
        "title": title,
        "description": description,
        "payment_methods": ["lightning", "credit_card"],
        'type': 'one-off'
    }]
    
    fs = Fewsats()
    r = fs.create_offers(offers_data)
    if not r.is_success:
        raise ValueError(f"Failed to create payment intent: {r.text}")
    
    return r

In [ ]:
# requires a FEWSATS_API_KEY environment variable
a.create_payment_intent( "12345", 1999,  title="Altra Shoes", description="Altra Escalanta v4 running shoes")


{'offers': [{'offer_id': '12345',
   'amount': 1999,
   'currency': 'USD',
   'description': 'Altra Escalanta v4 running shoes',
   'title': 'Altra Shoes',
   'payment_methods': ['lightning', 'credit_card'],
   'type': 'one-off'}],
 'payment_context_token': '00ce3d89-8313-4e39-a5a1-e21e2feab3c8',
 'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
 'version': '0.2.2'}

In [ ]:
data = {
  "title": "Eco-friendly Ball Point Pen",
  "amount": 22,
  "currency": "USD",
  "offer_id": "promotion-pen-22-cents",
  "description": "Promotion Gift Wholesale Cheap Eco-friendly Custom Logo Ball Point Pen"
}
a.create_payment_intent(**data)

{'offers': [{'offer_id': 'promotion-pen-22-cents',
   'amount': 22,
   'currency': 'USD',
   'description': 'Promotion Gift Wholesale Cheap Eco-friendly Custom Logo Ball Point Pen',
   'title': 'Eco-friendly Ball Point Pen',
   'payment_methods': ['lightning', 'credit_card'],
   'type': 'one-off'}],
 'payment_context_token': '4ad5d7c8-4485-4166-81ae-356725d9ec9a',
 'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
 'version': '0.2.2'}

In [ ]:
#| export

@patch
def as_tools(self:Agora):
    "Return list of available tools for AI agents"
    return [
        self.search_trial,
        self.get_product_detail,
        self.create_cart,
        self.add_to_cart,
        self.create_order,
        self.track_order,
        self.refresh_token,
        self.create_payment_intent
    ]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()